In [32]:
import requests
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter

In [33]:
def get_problems():
    url = "https://codeforces.com/api/problemset.problems"
    response = requests.get(url)
    data = response.json()
    if data["status"] != "OK":
        raise Exception("Error fetching problems: " + data.get("comment", "No comment provided"))
    # Return the list of problems
    return data["result"]["problems"]

In [34]:
def get_submissions(handle):
    url = f"https://codeforces.com/api/user.status?handle={handle}"
    response = requests.get(url)
    data = response.json()
    if data["status"] != "OK":
        raise Exception("Error fetching submissions: " + data.get("comment", "No comment provided"))
    return data["result"]

In [35]:
def get_contest_list():
    url = "https://codeforces.com/api/contest.list"
    response = requests.get(url)
    data = response.json()
    if data["status"] != "OK":
        raise Exception("Error fetching contest list: " + data.get("comment", "No comment provided"))
    contests = data["result"]
    # Build a mapping from contestId to contest name
    contest_mapping = {}
    for contest in contests:
        if "id" in contest and "name" in contest:
            contest_mapping[contest["id"]] = contest["name"]
    return contest_mapping

In [36]:
def process_data(handle):
    problems = get_problems()
    submissions = get_submissions(handle)
    contest_mapping = get_contest_list()
    
    submissions_by_contest = {}
    # Precalculate participation: contestId -> True if any submission has participantType "CONTESTANT"
    participation_by_contest = {}
    
    for sub in submissions:
        if "contestId" not in sub or "problem" not in sub:
            continue
        contestId = sub["contestId"]
        # Precalculate participation from author field.
        if "author" in sub and sub["author"].get("participantType") == "CONTESTANT":
            participation_by_contest[contestId] = True
        
        problem_index = sub["problem"]["index"]
        verdict = sub.get("verdict", "")
        sub_id = sub["id"]
        if contestId not in submissions_by_contest:
            submissions_by_contest[contestId] = {}
        if problem_index not in submissions_by_contest[contestId]:
            submissions_by_contest[contestId][problem_index] = {
                "verdict": verdict,
                "ok": verdict == "OK",
                "submissionId": sub_id
            }
        else:
            current = submissions_by_contest[contestId][problem_index]
            # If any submission is OK, record that.
            if verdict == "OK":
                current["ok"] = True
                current["verdict"] = "OK"
            elif not current["ok"] and sub_id > current["submissionId"]:
                current["submissionId"] = sub_id
                current["verdict"] = verdict

    # Start with contests where you have submissions.
    final_contest_ids = set(submissions_by_contest.keys())
    # If a contest's name includes "Div. 2" and contestId-1 exists with "Div. 1", include that contest.
    for cid in list(final_contest_ids):
        if cid in contest_mapping and "Div. 2" in contest_mapping[cid]:
            div1_cid = cid - 1
            if div1_cid in contest_mapping and "Div. 1" in contest_mapping[div1_cid]:
                final_contest_ids.add(div1_cid)
    
    # Map contestId to its list of problems.
    contests = {}
    for prob in problems:
        if "contestId" not in prob:
            continue
        contestId = prob["contestId"]
        if contestId in final_contest_ids:
            contests.setdefault(contestId, []).append(prob)
    
    return contests, submissions_by_contest, contest_mapping, participation_by_contest


In [37]:

def create_excel(contests, submissions_by_contest, contest_mapping, participation_by_contest, output_filename="Codeforces_Contest_Results.xlsx"):
    wb = Workbook()
    ws = wb.active
    ws.title = "Contest Results"
    
    # Define fill colors.
    green_fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")  # Light green for OK.
    yellow_fill = PatternFill(start_color="FFEB9C", end_color="FFEB9C", fill_type="solid")  # Light yellow for non-OK.
    
    row_idx = 1
    for contestId in sorted(contests.keys()):
        contest_name = contest_mapping.get(contestId, "Unknown Contest")
        contest_url = f"https://codeforces.com/contest/{contestId}"
        
        # Column A: Contest name (hyperlinked)
        contest_cell = ws.cell(row=row_idx, column=1, value=contest_name)
        contest_cell.hyperlink = contest_url
        contest_cell.style = "Hyperlink"
        
        # Column B: Participation indicator based on precalculated participation.
        participated = participation_by_contest.get(contestId, False)
        part_cell = ws.cell(row=row_idx, column=2, value="Participated" if participated else "")
        
        # Columns C onward: Problems
        col_idx = 3
        contest_problems = sorted(contests[contestId], key=lambda x: x["index"])
        for prob in contest_problems:
            problem_index = prob["index"]
            prob_url = f"https://codeforces.com/contest/{contestId}/problem/{problem_index}"
            prob_cell = ws.cell(row=row_idx, column=col_idx, value=problem_index)
            prob_cell.hyperlink = prob_url
            prob_cell.style = "Hyperlink"
            
            # Determine verdict for this problem.
            verdict = ""
            if contestId in submissions_by_contest and problem_index in submissions_by_contest[contestId]:
                verdict = submissions_by_contest[contestId][problem_index]["verdict"]
            
            # Apply fill based on verdict.
            if verdict == "OK":
                prob_cell.fill = green_fill
            elif verdict != "":
                prob_cell.fill = yellow_fill
            col_idx += 1
        row_idx += 1

    wb.save(output_filename)
    print(f"Excel file saved as {output_filename}")


In [38]:
def main():
    handle = "joseacaz"
    contests, submissions_by_contest, contest_mapping, participation_by_contest = process_data(handle)
    create_excel(contests, submissions_by_contest, contest_mapping, participation_by_contest)

if __name__ == "__main__":
    main()

Excel file saved as Codeforces_Contest_Results.xlsx
